In [2]:
import pandas as pd

In [3]:
def load_csv(file_path):
    df = pd.read_csv(file_path, header=None)
    return df

In [4]:
example = load_csv("../tests/refactoring.csv")

In [5]:
example

,0,1
0,568-862-697,(568)-862-697
1,123-567-890,(123)-567-890
2,421--134-143,(421)-134-143
3,341-134-536,NaN
4,254-245-455,NaN
5,(254)-245-254,NaN
6,657-456-356,NaN
7,365=356-324,NaN
8,189-248-823,NaN
9,819-459-159,NaN


<class 'str'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


In [5]:
valid = example[1].dropna()
len(valid)

3

In [6]:
from string import punctuation

In [7]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [36]:
def string_format(example, fill_col=2, cand=0):
    valid = example[fill_col].dropna()
    number = len(valid)
    start1 = str(example[0][0]).strip()
    start2 = str(valid[0]).strip()
    cands = ['extract', 'concat', 'refactoring', 'complex']
    pos = start1.find(start2)
    cand = 0 
    okay = True
    if pos != -1:
        context= [None, None]
        if pos==0:
            context[0]='start'
        else:
            context[0]=start1[pos-1]
        if pos+len(start2)==len(start1):
            context[1]='end'
        else:
            context[1]=start1[pos+len(start2)]  
        for i in range(1,number):
            start1 = example[0][i]
            start2 = valid[i]
            if context[0] == 'start':
                temp = start1[0:start1.find(context[1])]
                if temp != start2:
                    okay = False
                    break
            elif context[1] == 'end':
                temp = start1[start1.rfind(context[0])+1:]
                if temp != start2:
                    okay = False
                    break
            else:
                left = start1.find(context[0])
                right = start1.rfind(context[1])
                
                temp = start1[left+1:right]
                while temp != start2 and left < right:
                    right = start1.rfind(context[1], 0, right)
                    temp = start1[left+1:right]
                if temp == start2:
                    break
                right = start1.rfind(context[1])
                temp = start1[left+1:right]
                while temp != start2 and left < right:
                    left = start1.find(context[0], left+1)
                    temp = start1[left+1:right]
                if temp == start2:
                    break
                okay = False
                break
        if okay:
            return cands[cand], cand
    cand = 1 
    okay = True
    pos = start2.find(start1)
    if pos != -1:
        before = start2[:pos]
        after = start2[pos+len(start1):]
        for i in range(1, number):
            start1 = example[0][i]
            start2 = valid[i]
            if before+start1+after != start2 and start1 != start2:
                okay = False 
                break 
        if okay:
            return cands[cand], cand  
    cand = 2
    okay = True 
    origin = start1.lower()
    origin_lst = []
    goal = start2.lower()
    goal_lst = []
    ifpunc = False
    if goal[0] not in punctuation:
        goal_lst.append("")
        ifpunc = False
    else:
        goal_lst.append(goal[0])
        ifpunc = True
    for i in range(1,len(goal)):
        if goal[i] in punctuation and ifpunc:
            goal_lst[-1] += goal[i]
        elif goal[i] in punctuation and not ifpunc:
            goal_lst.append(goal[i])
            ifpunc = True
        elif goal[i] not in punctuation and ifpunc:
            ifpunc = False
    if not ifpunc: goal_lst.append("")
    
    for i in range(number):
        origin_lst = []
        origin = example[0][i]
        ifchar = False
        for char in origin:
            if char in punctuation and ifchar:
                ifchar = False
            elif char not in punctuation and not ifchar:
                origin_lst.append(char)
                ifchar = True
            elif char not in punctuation and ifchar:
                origin_lst[-1] += char
        print(goal_lst, origin_lst)
        assert(len(origin_lst)+1 == len(goal_lst))
        fused = ""
        for j in range(len(goal_lst)-1):
            fused += goal_lst[j]
            fused += origin_lst[j]
            
        fused += goal_lst[-1]
        print(valid[i], fused)
        if fused != valid[i]:
            okay = False
            break 
    if okay:
        return cands[cand], cand
        
    return None,-1

In [37]:
method, cand = string_format(example, fill_col=1, cand=0)

['(', ')-', '-', ''] ['568', '862', '697']
(568)-862-697 (568)-862-697
['(', ')-', '-', ''] ['123', '567', '890']
(123)-567-890 (123)-567-890
['(', ')-', '-', ''] ['421', '134', '143']
(421)-134-143 (421)-134-143


In [38]:
method

'refactoring'

In [26]:
cand

-1

In [27]:
"5".upper()

'5'